In [1]:
import os
os.chdir("/Users/maciej.filanowicz/Desktop/token_classification/src")

In [2]:
from hydra import initialize, compose
from hydra.utils import instantiate
from hydra.core.config_store import ConfigStore
from src.config import NERConfig
from src.data import DocumentWarehouse, NERDataset
from src.model import ModelTrainer
from torch.utils.data import DataLoader
import torch

In [3]:
cs = ConfigStore.instance()
cs.store(name='ner_config', node=NERConfig)

In [4]:
initialize(version_base="1.2", config_path="../conf", job_name="test_app")
cfg  = compose(config_name="config2", return_hydra_config=True)

In [5]:
model_handler = instantiate(cfg.model)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
device = torch.device('cpu')

In [7]:
dataset = DocumentWarehouse(cfg.dataset.images_dir,
                            cfg.dataset.annotations_dir).get_datasets(test_percentage=cfg.params.test_percentage,
                                                                      validation_percentage=cfg.params.validation_percentage)
train_dataset = NERDataset(dataset['train_dataset'], model_handler.processor, cfg.dataset.label2id)
test_dataset = NERDataset(dataset['test_dataset'], model_handler.processor, cfg.dataset.label2id)
validation_dataset = NERDataset(dataset['validation_dataset'], model_handler.processor, cfg.dataset.label2id)

In [8]:
training_dataloader = DataLoader(train_dataset, batch_size=cfg.params.batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=cfg.params.batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=cfg.params.batch_size)

In [9]:
# trainer = ModelTrainer(model=model_handler.model.to(device),
#                        device=device,
#                        training_dataloader=training_dataloader,
#                        validation_dataloader=validation_dataloader,
#                        test_dataloader=test_dataloader,
#                        epochs=cfg.params.epoch_count,
#                        use_wandb=True,
#                        project_name="NERv3",
#                        labels=list(cfg.dataset.label2id.keys()))
# trainer.run_trainer()